In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare
from models.Encoders import MHA_Encoder, FIM_Encoder, NPA_Encoder, Pipeline_Encoder
from models.SFI import SFI_gating

In [2]:
name='sfi'
hparams = {
    'scale':'demo',
    'mode':'train',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'value_dim':16,
    'query_dim':200,
    'head_num':16,
    'filter_num':150,
    'level':1,
    'epochs':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'k':10,
    'save_step':[0],
    'train_embedding':True,
    'news_id':True,
    'validate':False
}
torch.cuda.set_device(1)

In [3]:
vocab, loaders = prepare(hparams)

[2021-03-15 03:21:48,191] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'level': 1, 'epochs': 1, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'k': 10, 'save_step': [0], 'train_embedding': True, 'news_id': True, 'validate': False}
[2021-03-15 03:21:48,192] INFO (root) preparing dataset...
[2021-03-15 03:21:50,395] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'mha',hparams['select']])
mha_encoder = MHA_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, mha_encoder).to(hparams['device'])

In [ ]:
train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'fim',hparams['select']])

fim_encoder = FIM_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, fim_encoder).to(hparams['device'])

In [ ]:
train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'npa',hparams['select']])

hparams['user_dim'] = 200
hparams['query_dim'] = 200
hparams['filter_num'] = 400

npa_encoder = NPA_Encoder(hparams, vocab, len(loaders[0].dataset.uid2index))
sfi = SFI_gating(hparams, npa_encoder).to(hparams['device'])

In [ ]:
train(sfi, hparams, loaders, interval=10)

In [6]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'pipeline',hparams['select']])

hparams['pipeline'] = 'sfi-fim'
# hparams['scale'] = 'large'

ppl_encoder = Pipeline_Encoder(hparams)
sfi = SFI_gating(hparams, ppl_encoder).to(hparams['device'])

In [7]:
train(sfi, hparams, loaders, interval=10)

[2021-03-15 03:22:07,878] INFO (root) training...
epoch 1 , step 290 , loss: 1.5724: : 298it [00:10, 28.96it/s]
[2021-03-15 03:22:36,527] INFO (root) saved model of epoch 1
[2021-03-15 03:22:36,537] INFO (root) evaluating...
1815it [00:36, 49.46it/s]
[2021-03-15 03:23:14,077] INFO (root) evaluation results:{'auc': 0.5136, 'mean_mrr': 0.2199, 'ndcg@5': 0.2257, 'ndcg@10': 0.2969, 'step': 0}


SFI_gating(
  (encoder): Pipeline_Encoder(
    (news_repr): Embedding(101528, 150)
    (news_embedding): Embedding(101528, 9000)
  )
  (softmax): Softmax(dim=-1)
  (SeqCNN3D): Sequential(
    (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
  )
  (learningToRank): Linear(in_features=64, out_features=1, bias=True)
)